In [6]:
from ultralytics import YOLO
import cv2

# Version original

In [4]:
model = YOLO('yolov8n-face.pt')

img_path = 'face.jpg'

results = model(img_path)

boxes = results[0].boxes

img = cv2.imread(img_path)

for box in boxes:
    top_left_x = int(box.xyxy.tolist()[0][0])
    top_left_y = int(box.xyxy.tolist()[0][1])
    bottom_right_x = int(box.xyxy.tolist()[0][2])
    bottom_right_y = int(box.xyxy.tolist()[0][3])
    
    cv2.rectangle(img, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (50, 200, 129), 2)
    cv2.imwrite('face_output.jpg', img)

WARNING ⚠️ yolov8n-face.pt appears to require 'omegaconf', which is not in ultralytics requirements.
AutoInstall will run now for 'omegaconf' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['omegaconf'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 2.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=e4ce2b64c320d71663f4ac493aa9b58ed3cd22464ff870874502a56cdc01338e
  Stored in directory: /private/var/folders/lk/nng7lr0s295c52pz9q5fl45c0000gn/T/pip-ephem-

# Version live

In [7]:
import cv2

# Load YOLO model
model = YOLO('yolov8n-face.pt')

# Initialize the video capture object
cap = cv2.VideoCapture(0)  # 0 corresponds to the default camera (you can change it to a different camera index if necessary)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Perform face detection on the frame
    results = model(frame)

    # Get the bounding boxes for detected faces
    boxes = results[0].boxes if len(results) > 0 else []

    # Draw rectangles around detected faces
    for box in boxes:
        top_left_x, top_left_y, bottom_right_x, bottom_right_y = map(int, box.xyxy[0].tolist())
        cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (50, 200, 129), 2)

    # Display the resulting frame
    cv2.imshow('Face Detection', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 86.6ms
Speed: 7.6ms preprocess, 86.6ms inference, 7.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 56.9ms
Speed: 1.9ms preprocess, 56.9ms inference, 4.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 59.7ms
Speed: 1.6ms preprocess, 59.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 47.6ms
Speed: 1.7ms preprocess, 47.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 59.7ms
Speed: 1.6ms preprocess, 59.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 58.0ms
Speed: 2.3ms preprocess, 58.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 57.0ms
Speed: 2.8ms preprocess, 57.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 50.4ms
Speed: 1.8ms preprocess, 50.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

# Face recognition using DeepFace

In [1]:
import cv2
from deepface import DeepFace
from ultralytics import YOLO


# Load YOLO model for face detection
face_detection_model = YOLO('yolov8n-face.pt')

# Function to perform face recognition using DeepFace
def recognize_faces(frame):
    # Perform face detection using YOLO
    results = face_detection_model(frame)
    boxes = results[0].boxes if len(results) > 0 else []

    # Perform face recognition on detected faces using DeepFace
    for box in boxes:
        top_left_x, top_left_y, bottom_right_x, bottom_right_y = map(int, box.xyxy[0].tolist())
        face_roi = frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
        
        # Perform face recognition using DeepFace
        try:
            # Use DeepFace to verify if the detected face matches a known face
            result = DeepFace.verify("TheAlfredo.jpg", face_roi, model_name='VGG-Face', distance_metric='cosine')
            # If the similarity score is less than a threshold (e.g., 0.2), it's a match
            if result['distance'] < 0.2:
                cv2.putText(frame, "Matched", (top_left_x, top_left_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        except Exception as e:
            print("Error:", str(e))
        
        # Draw rectangle around detected face
        cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (50, 200, 129), 2)
    
    return frame

# Initialize the video capture object
cap = cv2.VideoCapture(0)  # 0 corresponds to the default camera (you can change it to a different camera index if necessary)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Perform face recognition on the frame
    frame_with_faces = recognize_faces(frame)

    # Display the resulting frame
    cv2.imshow('Face Recognition', frame_with_faces)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 44.2ms
Speed: 2.3ms preprocess, 44.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 42.9ms
Speed: 1.8ms preprocess, 42.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 face, 43.9ms
Speed: 1.8ms preprocess, 43.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


Error: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.



0: 384x640 1 face, 43.3ms
Speed: 1.7ms preprocess, 43.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


Error: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.



0: 384x640 1 face, 43.1ms
Speed: 1.9ms preprocess, 43.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)


...

Error: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.



0: 384x640 1 face, 44.1ms
Speed: 1.7ms preprocess, 44.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


Error: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.



0: 384x640 1 face, 44.1ms
Speed: 1.6ms preprocess, 44.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


Error: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


# Face recognition using FaceRecognition

In [5]:
import os
import cv2
import face_recognition

# Directory containing known face images
known_faces_dir = "/Users/alfredogarcia/Desktop/vs_code/IA_Avanzada/reto/face_recognition/faces/"

# Initialize lists to store known face encodings and labels
known_face_encodings = []
known_face_labels = []

# Load known face images and create face encodings
for filename in os.listdir(known_faces_dir):
    if filename.endswith(".jpg"):
        # Load the image file
        face_image = face_recognition.load_image_file(os.path.join(known_faces_dir, filename))

        # Extract face encodings from the image
        face_encoding = face_recognition.face_encodings(face_image)[0]  # Assuming there's only one face in each image

        # Extract the label (person's name) from the filename (excluding the extension)
        label = os.path.splitext(filename)[0]

        # Add the face encoding and label to the lists
        known_face_encodings.append(face_encoding)
        known_face_labels.append(label)

# Function to recognize faces in the frame
def recognize_faces(frame):
    # Find all face locations and face encodings in the current frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    # Iterate through each detected face and check if it matches a known face
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Check if the detected face matches any known face
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"  # Default label if no match is found

        # If a match is found, use the name corresponding to the known face
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_labels[first_match_index]

        # Draw rectangle around the face and display the name
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    return frame

# Rest of the code remains the same...
# Initialize the video capture object
cap = cv2.VideoCapture(0)  # 0 corresponds to the default camera (you can change it to a different camera index if necessary)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Perform face recognition on the frame
    frame_with_faces = recognize_faces(frame)

    # Display the resulting frame
    cv2.imshow('Face Recognition', frame_with_faces)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()